In [1]:
from helper_functions import *
from classes import *
from torch.utils.data import DataLoader
from torchvision import transforms
from timeit import default_timer as timer
from fold_functions import *
from plot_functions import *
import sklearn.model_selection

/home/theodoros/Projects/python/CNN-Classification/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the hyperparameters
DATA_DIR = 'data' # Path to the data directory
BATCH_SIZE = 32 # Batch size for the dataloaders
IN_CHANNELS = 3 # Number of input channels
HIDDEN_UNITS = 32  # Number of hidden units in the fully connected layer
NUM_CLASSES = 4 # Number of classes in the dataset
SIZE = 224 # Size of the images
LEARNING_RATE = 0.001 # Learning rate for the optimizer
EPOCHS = 250 # Number of epochs to train the model
GAMMA = 0.1 # Multiplicative factor of learning rate decay
STEP_SIZE = 15 # Step size for the learning rate scheduler
WEIGHT_DECAY = None # Weight decay for the optimizer
SEED = 1678737 # Seed for reproducibility
RANDOM_ROTATION = 20  # Random rotation for the images
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Create the dictionary that hold the hyperparameters
hyperparameters = {
        'BATCH_SIZE': BATCH_SIZE,
        'IN_CHANNELS': IN_CHANNELS,
        'HIDDEN_UNITS': HIDDEN_UNITS,
        'NUM_CLASSES': NUM_CLASSES,
        'SIZE': SIZE,
        'LEARNING_RATE': LEARNING_RATE,
        'EPOCHS': EPOCHS,
        'GAMMA': GAMMA,
        'STEP_SIZE': STEP_SIZE,
        'WEIGHT_DECAY': WEIGHT_DECAY,
        'SEED': SEED,
        'RANDOM_ROTATION': RANDOM_ROTATION,
        'DEVICE': DEVICE
    }

In [3]:
# Define the transforms
transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.RandomRotation(RANDOM_ROTATION),
    transforms.ToTensor(),
])

# Define the classes
classes = {
    'no_tumor': 0,
    'meningioma_tumor': 1,
    'pituitary_tumor': 2,
    'glioma_tumor': 3
}

In [4]:
walk_through_dir(DATA_DIR)

# Pre-Process the dataset
combined_dir = combine_and_rename_images(DATA_DIR,classes)

walk_through_dir(combined_dir)
# Split the dataset into train, validation, and test sets

There are 2 directories and 0 images in 'data'.
There are 4 directories and 0 images in 'data/Testing'.
There are 0 directories and 74 images in 'data/Testing/pituitary_tumor'.
There are 0 directories and 105 images in 'data/Testing/no_tumor'.
There are 0 directories and 100 images in 'data/Testing/glioma_tumor'.
There are 0 directories and 115 images in 'data/Testing/meningioma_tumor'.
There are 4 directories and 0 images in 'data/Training'.
There are 0 directories and 827 images in 'data/Training/pituitary_tumor'.
There are 0 directories and 395 images in 'data/Training/no_tumor'.
There are 0 directories and 826 images in 'data/Training/glioma_tumor'.
There are 0 directories and 822 images in 'data/Training/meningioma_tumor'.


Combining and renaming images: 100%|██████████| 2/2 [00:01<00:00,  1.46it/s]

Images combined and renamed successfully.
There are 0 directories and 3264 images in 'Combined'.


In [5]:
dataset_data, dataset_labels = load_data('Combined', transform=transform)
shutil.rmtree('Combined')

Loading images:   0%|          | 0/3264 [00:00<?, ?it/s]

Loading images: 100%|██████████| 3264/3264 [00:09<00:00, 336.21it/s]


In [6]:
import torch
from sklearn.model_selection import train_test_split

# Convert data and labels to PyTorch tensors
data_tensor = torch.stack(dataset_data).float() # Use torch.stack to convert the list of tensors to a single tensor. Call float() to cast to float32
labels_tensor = torch.tensor(dataset_labels, dtype=torch.long)

# Create the dataloaders
dataset_loader = DataLoader(CustomDataset(data_tensor, labels_tensor), batch_size=BATCH_SIZE, shuffle=True)

/home/theodoros/Projects/python/CNN-Classification/classes.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long) # List of labels


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
import numpy as np
from timeit import default_timer as timer

# Initialize KFold
n_splits = 5  # Number of folds
kf = KFold(n_splits=n_splits, shuffle=True)

# Placeholder for storing results
all_fold_results = []

# set seed
torch.manual_seed(SEED)

# Create the model and save the staring parameters
model = MRI_classification_CNN(IN_CHANNELS, HIDDEN_UNITS, NUM_CLASSES, SIZE, dropout=0.1)
start_params = model.state_dict()

# Perform K-Fold Cross-Validation
for fold, (train_idx, val_idx) in enumerate(kf.split(data_tensor)):
    print(f"Fold {fold + 1}/{n_splits}")

    # Split data
    train_data = data_tensor[train_idx]
    train_labels = labels_tensor[train_idx]
    val_data = data_tensor[val_idx]
    val_labels = labels_tensor[val_idx]

    # Create Dataloaders
    train_loader = DataLoader(CustomDataset(train_data, train_labels), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(CustomDataset(val_data, val_labels), batch_size=BATCH_SIZE, shuffle=False)

    # Initialize model to start params, optimizer, loss function, and scheduler
    model = MRI_classification_CNN(IN_CHANNELS, HIDDEN_UNITS, NUM_CLASSES, SIZE, dropout=0.2)
    model.load_state_dict(start_params)
    model.to(DEVICE)
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

    # Train the model
    start = timer()
    train_results = train(model, train_loader, val_loader, loss_fn, optimizer, EPOCHS, DEVICE, scheduler=scheduler, early_stopping=True)
    end = timer()

    # Test the model on the validation set
    test_loss, test_accuracy, test_preds, test_targets = test_step(model, val_loader, loss_fn, DEVICE, ep=0, EPOCHS=1)

    # Clear CUDA cache
    torch.cuda.empty_cache()

    # Save the results for each fold
    fold_results = {
        'fold': fold + 1,
        'train_time': end - start,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'preds': test_preds,
        'targets': test_targets,
        'train_results': train_results
    }
    all_fold_results.append(fold_results)

    print(f"Fold {fold + 1} results: test_loss: {fold_results['test_loss']}, test_acc: {fold_results['test_accuracy']*100.0}%")



Fold 1/5


[Epoch 12/250] Testing: 100%|██████████| 21/21 [00:16<00:00,  1.27it/s, Loss: 0.8760 | Accuracy: 79.1094%]


Saving model weights at epoch 12


[Epoch 14/250] Testing: 100%|██████████| 21/21 [00:16<00:00,  1.27it/s, Loss: 0.9348 | Accuracy: 79.2582%]


Early stopping at epoch 14


...

Saving model weights at epoch 137


[Epoch 138/250] Testing: 100%|██████████| 21/21 [00:14<00:00,  1.40it/s, Loss: 1.3861 | Accuracy: 81.3988%]


Saving model weights at epoch 138


[Epoch 140/250] Training:  62%|██████▏   | 51/82 [01:45<01:04,  2.07s/it, Loss: 0.0013 | Accuracy: 99.9387%] 


KeyboardInterrupt: 

In [ ]:
# Get the model with best accuracy
best_model_preds = []
best_model_targets = []
best_model_results = []

for fold_result in all_fold_results:
  if fold_result['test_accuracy'] == max(all_fold_results, key=lambda x: x['test_accuracy'])['test_accuracy']:
    best_model_preds = fold_result['preds']
    best_model_targets = fold_result['targets']
    best_model_results = fold_result['train_results']

In [ ]:
import numpy as np

# Convert to numpy arrays
best_model_preds = np.array(best_model_preds)
best_model_targets = np.array(best_model_targets)
y_true = best_model_targets
y_pred= best_model_preds

# # Convert continuous predictions to discrete class labels
# y_pred = np.argmax(y_pred_continuous, axis=1)

# Now you can compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix with sensitivity values
plot_confusion_matrix_with_sensitivity(cm, classes, normalize=True)

In [ ]:
# Plot loss and accuracy
plot_loss_curves(best_model_results)

In [ ]:
from torchvision import models
from torchsummary import summary

summary(model, (IN_CHANNELS, SIZE, SIZE))

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.metrics import roc_curve, roc_auc_score, recall_score, precision_score, f1_score
import numpy as np

# Inverting the dictionary to map from indices to class names
idx_to_class = {v: k for k, v in classes.items()}
all_preds = best_model_preds
all_labels = best_model_targets

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

print(f'all_preds shape: {all_preds.shape}')  # Should be (num_samples, NUM_CLASSES)
print(f'all_labels shape: {all_labels.shape}')  # Should be (num_samples,)

# Calculate ROC and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(all_labels == i, all_preds[:, i])
    roc_auc[i] = roc_auc_score(all_labels == i, all_preds[:, i])

# Calculate the sensitivity (recall) for each class
sensitivities = recall_score(all_labels, np.argmax(all_preds, axis=1), average=None)

# Calculate precision and F1 score for each class
precisions = precision_score(all_labels, np.argmax(all_preds, axis=1), average=None)
f1_scores = f1_score(all_labels, np.argmax(all_preds, axis=1), average=None)

# Printing the metrics with class names
for i in range(len(classes)):
    class_name = idx_to_class[i]
    print(f'Class {class_name} - AUC: {roc_auc[i]:.2f}, Precision: {precisions[i]:.2f}, '
          f'Recall (Sensitivity): {sensitivities[i]:.2f}, F1 Score: {f1_scores[i]:.2f}')

# Plotting the ROC curve for each class (optional)
import matplotlib.pyplot as plt

plt.figure()
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], lw=2, label='Class {0} (AUC = {1:0.2f})'.format(idx_to_class[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Multiclass')
plt.legend(loc="lower right")
plt.show()